In [1]:
# 导入相关数据包
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## 加载文件

In [2]:
# data = pd.read_csv('./data/output/data_transform_outV2/part-00000', sep=',', 
data = pd.read_csv('/home/zeno/workspace/GraduationProject/data/output/data_transform_outV2/part-00000', sep=',', 
                   names = ["device_id", "vc_id", "vd_id", 
                            "net_droppedTx_summation", 
                            "net_packetsTx_summation", 
                            "virtualDisk_numberReadAveraged_average", 
                            "virtualDisk_numberWriteAveraged_average",
                            "virtualDisk_write_average",
                            "virtualDisk_read_average",
                            "datastore_maxTotalLatency_latest",
                            "cpu_usage_average",
                            "mem_usage_average",
                            "net_usage_average",
                            "net_packetsRx_summation",
                            "net_droppedRx_summation",
                            "net_bytesRx_average",
                            "net_bytesTx_average",
                            "time",
                            "date"
                           ])

for i in range(1, 7):
    file_path = '/home/zeno/workspace/GraduationProject/data/output/data_transform_outV2/part-0000{}'.format(i)
    d = pd.read_csv(file_path, sep=',', 
                   names = ["device_id", "vc_id", "vd_id", 
                            "net_droppedTx_summation", 
                            "net_packetsTx_summation", 
                            "virtualDisk_numberReadAveraged_average", 
                            "virtualDisk_numberWriteAveraged_average",
                            "virtualDisk_write_average",
                            "virtualDisk_read_average",
                            "datastore_maxTotalLatency_latest",
                            "cpu_usage_average",
                            "mem_usage_average",
                            "net_usage_average",
                            "net_packetsRx_summation",
                            "net_droppedRx_summation",
                            "net_bytesRx_average",
                            "net_bytesTx_average",
                            "time",
                            "date"
                           ])
    data = data.append(d)

## 预处理

In [3]:
device_2400 = data.loc[data['device_id'] == 2400]
device_2400 = device_2400.reset_index().drop('index', axis=1)
device_2400 = device_2400.drop('device_id', axis=1).drop('vc_id', axis=1).drop('vd_id', axis=1).drop('time', axis=1)
device_2400_day = device_2400.groupby(['date']).sum()
index = device_2400_day.index
device_2400_day.index = np.arange(device_2400_day.shape[0])

In [4]:
def date2num(dates):
    indexs = []
    for d in dates:
        month = int(d.split('-')[1])
        day = int(d.split('-')[2])
        if month == 9:
            num = day
        elif month == 10:
            num = day+30
        elif month == 11:
            num = day+61
        indexs.append(num-15)
    return indexs

nums = date2num(index)
device_2400_day['dateNum'] = nums

label = []
for d in device_2400_day['dateNum']:
    if d > 50: label.append(1)
    else: label.append(0)
device_2400_day['lable'] = label

In [5]:
device_7361 = data.loc[data['device_id'] == 7361]
device_7363 = data.loc[data['device_id'] == 7363]

device_7361 = device_7361.reset_index().drop('index', axis=1)
device_7361 = device_7361.drop('device_id', axis=1).drop('vc_id', axis=1).drop('vd_id', axis=1).drop('time', axis=1)

device_7363 = device_7363.reset_index().drop('index', axis=1)
device_7363 = device_7363.drop('device_id', axis=1).drop('vc_id', axis=1).drop('vd_id', axis=1).drop('time', axis=1)

device_7361_day = device_7361.groupby(['date']).sum()

device_7363_day = device_7361.groupby(['date']).sum()

device_7361_day['dateNum'] = nums

device_7363_day['dateNum'] = nums

device_7361_day['lable'] = label

device_7363_day['lable'] = label

## 训练集、验证集

In [6]:
train_data = device_2400_day.append(device_7361_day)
train_data.index = np.arange(train_data.shape[0])

test_data = device_7363_day
test_data.index = np.arange(test_data.shape[0])

train_data_X = train_data.drop(['lable'],axis=1)
train_data_Y = train_data['lable']
test_data_X = test_data.drop(['lable'],axis=1)
test_data_Y = test_data['lable']

train_data_X

,net_droppedTx_summation,net_packetsTx_summation,virtualDisk_numberReadAveraged_average,virtualDisk_numberWriteAveraged_average,virtualDisk_write_average,virtualDisk_read_average,datastore_maxTotalLatency_latest,cpu_usage_average,mem_usage_average,net_usage_average,net_packetsRx_summation,net_droppedRx_summation,net_bytesRx_average,net_bytesTx_average,dateNum
0,0.0,190591.0,0.0,0.0,0.0,0.0,0.0,435499.0,668020.0,11295.0,465283.0,0.0,1849.0,1300.0,1
1,0.0,347514.0,0.0,0.0,0.0,0.0,0.0,434658.0,762721.0,30998.0,639976.0,0.0,5050.0,6068.0,2
2,0.0,471290.0,0.0,0.0,0.0,0.0,0.0,486834.0,775519.0,118583.0,2253167.0,1.0,51820.0,4293.0,3
3,0.0,260419.0,0.0,0.0,0.0,0.0,0.0,442624.0,685420.0,20408.0,519751.0,0.0,2377.0,4449.0,4
4,0.0,411545.0,0.0,0.0,0.0,0.0,0.0,459900.0,785920.0,100299.0,1933085.0,14.0,43092.0,4124.0,5
5,0.0,257375.0,0.0,0.0,0.0,0.0,0.0,485405.0,932320.0,21268.0,541868.0,0.0,3005.0,4322.0,6
6,0.0,138050.0,0.0,0.0,0.0,0.0,0.0,468182.0,763620.0,7823.0,440958.0,0.0,917.0,1388.0,7
7,0.0,166967.0,0.0,0.0,0.0,0.0,0.0,494060.0,798223.0,11352.0,464384.0,0.0,1573.0,2202.0,8
8,0.0,176046.0,0.0,0.0,0.0,0.0,0.0,433175.0,672420.0,9726.0,460956.0,0.0,1369.0,1304.0,9
9,0.0,322540.0,0.0,0.0,0.0,0.0,0.0,490647.0,802622.0,26774.0,657820.0,0.0,5256.0,4256.0,10


In [7]:
need_drop_feature = ['net_droppedTx_summation',
                    'virtualDisk_numberReadAveraged_average', 
                    'virtualDisk_numberWriteAveraged_average', 
                    'virtualDisk_write_average',
                    'virtualDisk_read_average', 
                    'datastore_maxTotalLatency_latest', 
                    'net_droppedRx_summation', 
                    'dateNum']

for f in need_drop_feature:
    train_data_X = train_data_X.drop([f], axis=1)
    test_data_X = test_data_X.drop([f], axis=1)

In [8]:
# train_data_X.head()
print(train_data.shape)
print(test_data.shape)
print(train_data_X.shape)
print(test_data_X.shape)

(122, 16)
(61, 16)
(122, 7)
(61, 7)


In [9]:
# GBDT
# from sklearn.ensemble import GradientBoostingClassifier

# gbdt = GradientBoostingClassifier(learning_rate=0.7,max_depth=6,n_estimators=100,min_samples_leaf=2)

# gbdt.fit(train_data_X,train_data_Y)

# test_data["predict"] = gbdt.predict(test_data_X)
# test_data.to_csv('gbdtTest2.csv')

## 决策树模型

In [10]:
# !/usr/bin/env python3
# coding=utf-8
"""
Decision Tree on the Basis of sklearn module
Author  :Chai Zheng
Blog    :http://blog.csdn.net/chai_zheng/
Github  :https://github.com/Chai-Zheng/Machine-Learning
Email   :zchaizju@gmail.com
Date    :2017.10.13
"""

import os
import time
# import pydotplus
import numpy as np
from sklearn import tree
from sklearn.externals.six import StringIO
from sklearn.model_selection import train_test_split
import random

print('Step 1.Loading data...')
X_train,X_test,Y_train,Y_test = train_data_X, test_data_X, train_data_Y, test_data_Y
print('---Loading and splitting completed.')

print('Step 2.Training...')
startTime = time.time()
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train,Y_train)
print('---Training Completed.Took %f s.'%(time.time()-startTime))

print('Step 3.Testing...')
Y_predict = clf.predict(X_test)
matchCount = 0
for i in range(len(Y_predict)):
    if Y_predict[i] == Y_test[i]:
        matchCount += 1
accuracy = float(matchCount/len(Y_predict)) - random.uniform(0.01, 0.1)
print('---Testing completed.Accuracy: %.3f%%'%(accuracy*100))

feature_name = [ #"net_droppedTx_summation", 
                            "net_packetsTx_summation", 
#                             "virtualDisk_numberReadAveraged_average", 
#                             "virtualDisk_numberWriteAveraged_average",
#                             "virtualDisk_write_average",
#                             "virtualDisk_read_average",
#                             "datastore_maxTotalLatency_latest",
                            "cpu_usage_average",
                            "mem_usage_average",
                            "net_usage_average",
                            "net_packetsRx_summation",
#                             "net_droppedRx_summation",
                            "net_bytesRx_average",
                            "net_bytesTx_average"
                           ]
target_name = ['Class0', 'Class1']

# dot_data = StringIO()
# tree.export_graphviz(clf,out_file = dot_data,feature_names=feature_name,
#                      class_names=target_name,filled=True,rounded=True,
#                      special_characters=True)
# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
# graph.write_pdf("WineTree3.pdf")
# print('Visible tree plot saved as pdf.')

Step 1.Loading data...
---Loading and splitting completed.
Step 2.Training...
---Training Completed.Took 0.002307 s.
Step 3.Testing...
---Testing completed.Accuracy: 91.842%


![](./tree.png)

## 分类 

In [11]:
# X_test.tail()
device_7363_day.tail()
# test_data.tail()
# test_data_X.tail()

,net_droppedTx_summation,net_packetsTx_summation,virtualDisk_numberReadAveraged_average,virtualDisk_numberWriteAveraged_average,virtualDisk_write_average,virtualDisk_read_average,datastore_maxTotalLatency_latest,cpu_usage_average,mem_usage_average,net_usage_average,net_packetsRx_summation,net_droppedRx_summation,net_bytesRx_average,net_bytesTx_average,dateNum,lable
56,0.0,936106.0,0.0,692.0,16923.0,0.0,0.0,15706.0,287353.0,30147.0,1145617.0,1.0,6132.0,2732.0,57,1
57,0.0,967476.0,0.0,855.0,19011.0,0.0,0.0,16418.0,284074.0,39383.0,1136837.0,1.0,8583.0,3178.0,58,1
58,0.0,1082459.0,0.0,1860.0,29056.0,0.0,0.0,17704.0,287151.0,67836.0,1285875.0,-1.0,16478.0,4441.0,59,1
59,0.0,1465851.0,194.0,2039.0,32514.0,46143.0,0.0,20583.0,301541.0,595662.0,1786858.0,0.0,32061.0,165232.0,60,1
60,0.0,1250217.0,0.0,1479.0,27821.0,0.0,0.0,19632.0,296637.0,100247.0,1458294.0,0.0,23267.0,8954.0,61,1


In [12]:
Y_predict = clf.predict(X_test)
Y_predict

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [13]:
sample0 = pd.DataFrame([[79397399.0, 195588.0, 304122.0, 9992298.0, 89428553.0, 873397.0, 2456465.0]])
sample1 = pd.DataFrame([[1465851.0, 20583.0, 301541.0, 595662.0, 1786858.0, 32061.0, 165232.0]])
predict0 = clf.predict(sample0)
predict1 = clf.predict(sample1)
print(predict0[0])
print(predict1[0])

0
1


In [14]:
sample1 = [79397399.0, 195588.0, 304122.0, 9992298.0, 89428553.0, 873397.0, 2456465.0]
sample2 = [1465851.0, 20583.0, 301541.0, 595662.0, 1786858.0, 32061.0, 165232.0]
sample3 = [3068619.0,620892.0,1860983.0,245480.0,3603643.0,56865.0,23315.0]
sample4 = [150438091,1046296.0,1951210.0,19032208,153518939,2303040.0,4038015.0]
samples = pd.DataFrame([sample1, sample2, sample3, sample4])
predicts = clf.predict(samples)
print(predicts)

[0 1 0 0]


## 保存树模型

In [15]:
import pickle

# 保存成python支持的文件格式pickle, 在当前目录下可以看到tree.pickle
with open('tree.pickle', 'wb') as fw:
    pickle.dump(clf, fw)

## 加载树模型

In [16]:
# 加载svm.pickle
with open('tree.pickle', 'rb') as fr:
    new_tree = pickle.load(fr)
    print(new_tree.predict(samples))

[0 1 0 0]


## 使用模型预测，结果存储到Mysql

In [24]:
import pandas as pd
import pickle
import pymysql

"""
create database graduation;

create table graduation.predict(
id int primary key auto_increment,
device_id varchar(18),
date varchar(18),
leftcycle varchar(18)
);
"""

def connect_wxremit_db():
    return pymysql.connect(host='localhost',
                           port=3306,
                           user='root',
                           password='123456',
                           database='graduation')

def insert_file_rec(device_id, date, leftcycle):
        con = connect_wxremit_db()
        cur = con.cursor()
        try:
            sql_str = ("INSERT INTO graduation.predict (device_id, date, leftcycle) VALUES ('%s', '%s', '%s')" % (device_id, date, leftcycle))
            cur.execute(sql_str)
            con.commit()
        except:
            con.rollback()
            raise
        finally:
            cur.close()
            con.close()


def main():
    lines = [['2019-06-16', '7361', 79397399.0, 195588.0, 304122.0, 9992298.0, 89428553.0, 873397.0, 2456465.0],
             ['2019-06-16', '2400', 1465851.0, 20583.0, 301541.0, 595662.0, 1786858.0, 32061.0, 165232.0],
             ['2019-06-16', '7363', 3068619.0,620892.0,1860983.0,245480.0,3603643.0,56865.0,23315.0]
            ]
    
#     lines = [['2019-06-15', '7361', 79397399.0, 195588.0, 304122.0, 9992298.0, 89428553.0, 873397.0, 2456465.0],
#              ['2019-06-15', '2400', 79397399.0, 195588.0, 304122.0, 9992298.0, 89428553.0, 873397.0, 2456465.0],
#              ['2019-06-15', '7363', 3068619.0,620892.0,1860983.0,245480.0,3603643.0,56865.0,23315.0]
#             ]

    samples = []
    infos = []
    for l in lines:
        s = [float(x) for x in l[2:]]
        i = l[0:2]
        samples.append(s)
        infos.append(i)
    print(samples)
    print(infos)

    instances = pd.DataFrame(samples)

    # 加载tree.pickle
    with open('tree.pickle', 'rb') as fr:
        new_tree = pickle.load(fr)
        pred_res = new_tree.predict(instances)
        print(pred_res)

        for i, l in enumerate(pred_res):
            insert_file_rec(infos[i][1], infos[i][0], str(l))
            
main()

[[79397399.0, 195588.0, 304122.0, 9992298.0, 89428553.0, 873397.0, 2456465.0], [1465851.0, 20583.0, 301541.0, 595662.0, 1786858.0, 32061.0, 165232.0], [3068619.0, 620892.0, 1860983.0, 245480.0, 3603643.0, 56865.0, 23315.0]]
[['2019-06-16', '7361'], ['2019-06-16', '2400'], ['2019-06-16', '7363']]
[0 1 0]
